In [1]:
import pandas as pd
import sqlite3

# Connect to aia_big.db and tell what data is there
def connect_to_database(db_path='weekly/db/aia_big.db'):
    """
    Connect to the SQLite database and return the connection object.
    
    Parameters:
    -----------
    db_path : str, default='aia_big.db'
        Path to the SQLite database file.
    """

    conn = sqlite3.connect(db_path)
    return conn

def get_table_info(conn):
    """
    Get information about the tables in the database.
    
    Parameters:
    -----------
    conn : sqlite3.Connection
        Connection object to the SQLite database.
    
    Returns:
    --------
    list of tuples containing table names and their row counts.
    """
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    table_info = []
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
        row_count = cursor.fetchone()[0]
        table_info.append((table[0], row_count))
    
    return table_info

def fetch_table_data(conn, table_name):
    """
    Fetch data from a specific table in the database.
    
    Parameters:
    -----------
    conn : sqlite3.Connection
        Connection object to the SQLite database.
    table_name : str
        Name of the table to fetch data from.
    
    Returns:
    --------
    pandas DataFrame containing the table data.
    """
    query = f"SELECT * FROM {table_name}"
    return pd.read_sql_query(query, conn)

def close_database_connection(conn):
    """
    Close the database connection.
    
    Parameters:
    -----------
    conn : sqlite3.Connection
        Connection object to the SQLite database.
    """
    conn.close()

conn = connect_to_database()
instrument = input("Enter instrument name to fetch data (e.g., 'USD_CAD'): ") or 'USD_CAD'
rows = int(input("Enter number of rows to fetch (e.g., 10000): ") or 100000000)
offset = int(input("Enter offset in hours (e.g., 0): ") or 4)

# query timestamp with n hours of offset on the date
query = f"SELECT datetime(date, '-{offset} hours') AS timestamp, ROUND(buy, 5) AS price FROM gather WHERE instrument = '{instrument}' ORDER BY timestamp LIMIT {rows}"
df = pd.read_sql_query(query, conn)
print(f'{len(df)} rows fetched from database for instrument {instrument} with offset {offset} hours')
close_database_connection(conn)

df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.drop_duplicates(subset=['timestamp'], keep='last')


# Remove rows where for a given day there is less than 12 rows
df = df.groupby(df['timestamp'].dt.date).filter(lambda x: len(x) >= 12)

# Resample to 5-second intervals, forward filling missing values
df.set_index('timestamp', inplace=True)
df = df.resample('1s').ffill().reset_index()

df = df.dropna(subset=['price'])

# ordder by timestamp
df = df.sort_values(by='timestamp').reset_index(drop=True)

# remove 10 fist rows
df = df.iloc[10:]

original_df = df.copy()

len(df)


335143 rows fetched from database for instrument USD_CAD with offset 4 hours


431605

In [ ]:
df = original_df.copy()

In [2]:
# ask for a weekday number and keep the data for that day
weekday_number = input("Enter weekday number (0=Monday, 6=Sunday): ") or 0
weekday_number = int(weekday_number)
df = df[df['timestamp'].dt.weekday == weekday_number]
print(df.head())
print(df.tail())
print(len(df))

                 timestamp    price
370505 2025-07-11 00:00:00  1.36931
370506 2025-07-11 00:00:01  1.36931
370507 2025-07-11 00:00:02  1.36931
370508 2025-07-11 00:00:03  1.36931
370509 2025-07-11 00:00:04  1.36932
                 timestamp    price
431610 2025-07-11 16:58:25  1.36927
431611 2025-07-11 16:58:26  1.36927
431612 2025-07-11 16:58:27  1.36927
431613 2025-07-11 16:58:28  1.36927
431614 2025-07-11 16:58:29  1.36931
61110


In [3]:
from algo import *
from oanda import *
from helper import *
from live_graph_server import run_server, graph_updater

precision = 5
purple = Algo(base_interval='15min', slow_interval='30min',aspr_interval='3min', peak_interval='2min')

# Start the Flask server in a background thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Process historical data in batches to reduce memory usage
batch_size = 100
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    
    for _, row in batch.iterrows():
        timestamp = row['timestamp']
        price = round(row['price'], precision)
        
        # Process the historical data row with the Algo instance
        take, return_dict = purple.process_row(timestamp, price, precision, say=False)
        
        # Update the live graph with historical data
        graph_updater.update_graph(return_dict)

🌐 Starting Flask server...1.36931000 1.36937000
0.00004382
2025-07-11 00:00:23 - Price: 1.36938, EMA: 1.36931, TEMA: 1.36932, E base_mamplitude: False, Cross Direction: 1

1.36973000 1.36940000
0.00024098
1.36936000 1.36949000
0.00009493
1.36949000 1.36989000
0.00029199
1.36988000 1.36990000
0.00001460
1.36993000 1.37001000
0.00005839
 * Serving Flask app 'live_graph_server'
1.37005000 1.37009000
0.00002920
1.37023000 1.37014000
0.00006569
1.37014000 1.37012000
0.00001460
1.37009000 1.37023000
0.00010217
1.37022000 1.37022000
0.00000000
1.37025000 1.37010000
0.00010948
1.37010000 1.37010000
0.00000000
2025-07-11 00:39:02 - Price: 1.36993, EMA: 1.37003, TEMA: 1.37002, E base_mamplitude: True, Cross Direction: -1
1.36982000 1.36994000
0.00008760
1.36994000 1.36996000
0.00001460
1.36992000 1.37001000
0.00006569
1.36994000 1.36994000
0.00000000
1.37014000 1.36994000
0.00014599
1.36981000 1.36995000
0.00010219
1.36998000 1.36981000
0.00012410
1.36974000 1.36991000
0.00012410
1.36991000 1.36

In [ ]:
# stop the server thread
server_thread.join(timeout=1)
if server_thread.is_alive():
    print("Server thread did not stop gracefully, terminating.")
else:
    print("Server thread stopped gracefully.")